# Coleta de vagas do Linkedin

In [1]:
import time
from datetime import datetime

import pandas as pd
import requests
from bs4 import BeautifulSoup

**Funções auxiliares:**

In [2]:
def fetch_page(url, verbose=False):
    """
    Fetches a web page with retries and exponential backoff.
    Args:
        url (str): The URL of the web page to fetch.
        verbose (bool): If True, prints status messages.
    Returns:
        str: The content of the web page, or None if failed.
    """
    wait_time = 1

    while True:
        if wait_time > 60:
            if verbose:
                print(f"Falha ao fazer o fetch de {url} depois de várias tentativas.")
            return None

        try:
            response = requests.get(url)
            # Forçando um erro se a resposta não for bem-sucedida
            response.raise_for_status()
            if verbose:
                print(f"Fetch da {url} feito com sucesso")
            return response.text
        except requests.exceptions.RequestException as e:
            if verbose:
                print(
                    f"Erro ao fazer o fetch de {url}: {e}. Tentando novamente em {wait_time} segundos..."
                )

            time.sleep(wait_time)
            # Backoff exponencial
            wait_time *= 2

In [3]:
def fetch_job_details(job_id, is_remote, verbose=False):
    """
    Fetches job details from LinkedIn job posting.
    Args:
        job_id (str): The ID of the job to fetch.
        is_remote (bool): Whether the job is remote.
        verbose (bool): If True, prints status messages.
    Returns:
        dict: A dictionary containing job details, or None if failed.
    """
    job_url = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"

    job_data = fetch_page(job_url, verbose)
    # Se a vaga não foi encontrada ou ocorreu um erro, ignorar ela
    if not job_data:
        return None

    # Fazer o parsing do HTML da vaga
    soup = BeautifulSoup(job_data, "html.parser")

    # Informações sobre a vaga
    job_name = soup.select_one(".top-card-layout__entity-info a")
    job_location = soup.select_one(".topcard__flavor-row .topcard__flavor--bullet")
    job_description = soup.select_one(".description__text")
    job_remote = is_remote

    # Ignorando a vaga se alguma informação estiver faltando
    if not job_name or not job_location or not job_description:
        if verbose:
            print(f"Pulando vaga {job_id} devido a informações faltantes.")
        return None

    # Criando um dicionário para armazenar os detalhes da vaga
    job_details = {
        "id": job_id,
        "name": job_name.get_text(strip=True),
        "location": job_location.get_text(strip=True),
        "description": job_description.get_text(strip=True),
        "remote": job_remote,
    }

    return job_details

In [4]:
job_location = "Brazil"
maximum_time = "r86400"  # considerando apenas vagas das últimas 24 horas


def fetch_jobs_with_keywords(keywords, verbose=False):
    """
    Fetches job details from LinkedIn using a certain keyword. Considers both remote and non-remote jobs.
    Args:
        keywords (str): The keywords to search for.
        verbose (bool): If True, prints status messages.
    Returns:
        list: A list of dictionaries containing job details, or an empty list if nothing was found.
    """
    all_jobs = []

    base_search_url = (
        "https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search"
    )

    keywords = keywords.replace(" ", "%20")

    # Buscando vagas remotas e não remotas
    for is_remote in [False, True]:
        if verbose:
            print(
                f"Buscando vagas {'remotas' if is_remote else 'não remotas'} para a palavra-chave: {keywords}"
            )

        # Adicionando palavra-chave, localização, parâmetro de tempo e filtro remoto/não-remoto
        location = job_location
        time = maximum_time
        remote = "2" if is_remote else "1%2C3"

        search_url = f"{base_search_url}?keywords={keywords}&location={location}&f_TPR={time}&f_WT={remote}&start={{}}"

        pagination_index = 0
        while True:
            jobs = fetch_page(search_url.format(pagination_index), verbose)

            # Se houve um erro ao buscar a página, parar a busca
            if not jobs:
                break

            # Fazendo o parse das vagas
            soup = BeautifulSoup(jobs, "html.parser")

            # Verificando se não há vagas encontradas
            total_jobs = soup.find_all("li")
            if not total_jobs:
                if verbose:
                    print(f"Não há mais vagas na página {pagination_index}.")
                break

            # Extraindo detalhes da vaga para cada vaga
            for job in total_jobs:
                job_id = (
                    job.find("div", {"class": "base-card"})
                    .get("data-entity-urn")
                    .split(":")[3]
                    if job.find("div", {"class": "base-card"})
                    else None
                )

                if job_id is None:
                    # Não há mais vagas para processar
                    break

                job_information = fetch_job_details(job_id, is_remote, verbose)

                # Adicionando a vaga se todas as informações foram obtidas
                if job_information:
                    all_jobs.append(job_information)

            # Passando para a próxima página
            pagination_index += len(total_jobs)

    if verbose:
        print(f"Total de vagas obtidas: {len(all_jobs)}")

    return all_jobs

**Palavras-chave para as buscas:**

In [5]:
# Keywords de busca
keywords = [
    "Desenvolvedor",
    "Programador",
    "Software",
    "Hardware",
    "IA",
    "Inteligência Artificial",
    "Machine Learning",
    "Data Science",
    "Engenheiro de Software",
    "Engenheiro de Dados",
    "Desenvolvimento Web",
    "Backend",
    "Frontend",
    "Full Stack",
    "Cloud",
    "DevOps",
    "Big Data",
    "QA",
    "UX",
    "UI",
    "CI",
    "CD",
    "Android",
    "iOS",
    "Mobile",
    "TI",
    "Cibersegurança",
    "Redes",
    "Robótica",
    "Jogos",
]

**Coleta de dados usando as palavras-chave definidas:**

In [6]:
verbose = False

# Buscando vagas para cada palavra-chave e salvando tudo em um arquivo CSV
all_jobs_data = []
for keyword in keywords:
    jobs = fetch_jobs_with_keywords(keyword, verbose)

    if jobs:
        all_jobs_data.extend(jobs)

# Convertendo em um DataFrame, removendo possíveis vagas duplicadas e salvando como CSV
if all_jobs_data:
    df = pd.DataFrame(all_jobs_data)
    # Busca do LinkedIn: vagas podem aparecer em diferentes palavras-chave
    df = df.drop_duplicates()

    csv_filename = f"linkedinjobs_{datetime.today().strftime('%Y-%m-%d')}.csv"
    df.to_csv(csv_filename, index=False)

    print(f"Foram encontradas e salvas {len(all_jobs_data)} vagas")

Foram encontradas e salvas 3290 vagas
